# Washington DC Housing Market Analysis

## Project Goal

## Summary of Data


This analysis uses DC housing market data from Refin for condos and macroeconomic data from Fred with the time periods between February 2010 and October 2019. Housing data includes data for prices (median sale price, percentage of homes sold above list price, percentage of homes that had price drop, etc.), inventory (number of homes on market, new listings, months of supply, etc.), and sales (number of homes sold, median days on market, etc.). For economic data, refer to [readme](https://github.com/iuniorhsiung/mod4_project_DC_housing_price/blob/master/data/readme.md). Most of our economic data are regional data in DC or DC-VA-MD-WV regions. 

## Methodology

In order to price the condos in Washington DC, we perform time-series linear regression with DC housing statistics and regional macroeconomic variables. For the theory behind it, please refer to [this](https://www.reed.edu/economics/parker/312/tschapters/S13_Ch_2.pdf).


### Data Analysis

After collecting our data, based on the model assumptions, we need to comfirm if both independent and dependent variables are stationary. In addition, we check if the autocorrelation for these variables is zero. There are four parts are performed: (1) time series plots, (2) acf/pacf plots, (3) [stationary test](https://en.wikipedia.org/wiki/KPSS_test) and (4) [autocorrelation test](https://en.wikipedia.org/wiki/Durbin%E2%80%93Watson_statistic). For the positive variables, we will also use log-transformation for them. In addition, based on the dataset, we construct a 1st difference transfomation for all features as our final dataset.   

Note that our data time frame ranges from Feburary 2010 to October 2019. It includes 117 monthly data. 


### Feature Selection 

Based on the data analysis section, we pre-selected a pool of our features (independent variables). In this section, we perform (1) [Forward-Backward stepwise regression](https://en.wikipedia.org/wiki/Stepwise_regression) and (2) [KBest selection](https://www.kaggle.com/jepsds/feature-selection-using-selectkbest)

###  Model Fit

We perform a time-series regression based on the selected features in Feature Selection section. Note that we will check if the sign for each feature comes with business rationale. For our dataset, we divide it into train dataset/In-Time Sample and test dataset/Out-of-Time Sample (OOT Sample). For the time-series modeling practice, we use the first 100 monthly data as In-Time sample and last 17 monthly data as OOT sample. 


### Cross Validation

We will work on cross validaiton on OOT sample since there is a time effect in our housing dataset. 


### Model Performance

We evaluate our model performance based on the following two metrics:

- Mean Absolute Error (MAE): Based on our project goal, MAE estimates the dollar impact for the model residual. For example, it is 700 dollars in error if MAE is 700. Please refer to [MAE].(https://en.wikipedia.org/wiki/Mean_absolute_error)

- Mean Absolute Percentage Error (MAPE): Similar to MAE, MAPE esitmates the percentage impact in housing price. For example, the model shows about 2.5% error if MAPE is .0249. Please refer to [MAPE].(https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)

### Residual Analysis

Based on the assumptions for [time series linear regression](https://www.reed.edu/economics/parker/312/tschapters/S13_Ch_2.pdf) in Methodology section, we perform the following analyses:

- Stationary test: To ensure the constant mean exists for model residual.

- Homoscedastic test: To ensure the constant variance for model residual. We perform both [Breusch–Pagan test](https://en.wikipedia.org/wiki/Breusch%E2%80%93Pagan_test) and [Goldfeld–Quandt test](https://en.wikipedia.org/wiki/Goldfeld%E2%80%93Quandt_test). 

- Autocorrelation test: To ensure there is no strong autocorrelation in moder residual (or autocorrelation is zero). 

In addition, we study the distribution of the model residuals with the following analyses:

- Normality test: We will perform both normality test based on [D’Agostino and Pearson’s test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html) and [KPSS test](https://en.wikipedia.org/wiki/KPSS_test).

- Q-Q plot



### Rationale for the current approach:



### Data Import

Here, we import all code and libraries based on the "Data_cleaning_house_price.ipynb" and "ConstructingDataFrame.ipynb". This takes care of all data imports and data format for the final modeling use including transformed dataset.

In [ ]:
# %load ../python_folder/python_files/importlib

### Data Transformation

We perform two types of transformation: log, first difference

In [ ]:
# %load ../python_folder/python_files/transformation

### Data visualization

In Data Analysis section, time series plots and ACF/PACF plots are used to check the data visually. Please refer to [DC_house_price_EDA_JC.ipynb](https://github.com/iuniorhsiung/mod4_project_DC_housing_price/tree/master/master_notebook)

### Hypothesis Test

We performed the following hypothesis tests on our housing/economic dataframe:

- KPSS test for stationarity 
- ADF test for stationarity
- Durbin–Watson for autocorrelation

In [ ]:
# Code for Hypothesis Test
# %load ../python_folder/python_files/test_matrix_kpss_dw

In [ ]:
# Run hypothesis tests for df with log transformation
test_matrix = hypo_test(col_name = col_name, data = df)
# Run hypothesis tests for df1 with 1st diff transformation
test_matrix1 = hypo_test(col_name = col_name1, data = df1)

In [ ]:
# Code for determine sig level and index
# %load ../python_folder/python_files/Test_p_value

#### df dataset
Original Data with log transformation 

In [ ]:
test_p_value(p = .05, name = 'adftest')
test_p_value(p = .05, name = 'kpsstest')
test_p_value(p = .05, name = 'ljtest')
test_matrix
# Save the final testing results in data folder
test_matrix.to_csv('../data/TM_test.csv')

In [ ]:
# Find all features are stationary based on both KPSS and adf tests
index_adf = test_matrix['index_adftest'] == 1
index_kpss = test_matrix['index_kpsstest'] == 0
index_lj = test_matrix['index_ljtest'] == 0
index_dw1 = test_matrix['stat_dw'] >= .9
index_dw2 = test_matrix['stat_dw'] <= 3.1 
test_col = list(test_matrix[index_adf & index_kpss]["Features"])
test_col
test_col.remove('Median Sale Price MoM')

#### df1 dataset
Data with 1st diff transformation 

In [ ]:
test_p_value(p = .05, name = 'adftest', data = test_matrix1)
test_p_value(p = .05, name = 'kpsstest', data = test_matrix1)
test_p_value(p = .05, name = 'ljtest', data = test_matrix1)
test_matrix1
test_matrix1.to_csv('../data/TM_test_1diff.csv')

In [ ]:
# Find all features are stationary based on both KPSS and adf tests
index_kpss = test_matrix1['index_kpsstest'] == 0
index_lj = test_matrix1['index_ljtest'] == 0
index_dw1 = test_matrix1['stat_dw'] >= .9 
index_dw2 = test_matrix1['stat_dw'] <= 3.1 
index_adf = test_matrix1['index_ljtest'] == 1
test_col1 = list(test_matrix1[index_adf & index_kpss & index_dw1 & index_dw2]["Features"])
test_col1
test_col1.remove('Median Sale Price_1diff')
test_col1.remove('Median Sale Price_log_1diff')
test_col1.remove('Median Sale Price MoM_1diff')

#### Split In-Time and Out-of-Time samples 

In [ ]:
train_x = pd.concat([df[test_col][1:100], df1[test_col1][0:99]], axis=1)
test_x = pd.concat([df[test_col][100:117], df1[test_col1][99:116]], axis=1)
train_y = df1["Median Sale Price_1diff"][0:99]
test_y = df1["Median Sale Price_1diff"][99:116]

### Feature Selection: Forward/Backward stepwise regression and KBest

In [ ]:
# Code for Stepwise Regression
# %load ../python_folder/python_files/ForwardBackwardStepwiseRegression

In [ ]:
result = stepwise_selection(train_x, train_y, threshold_in = 0.1, threshold_out = 0.15)

In [ ]:
print('resulting features:')
print(result)

In [ ]:
# %load ../python_folder/python_files/KBest
# Feature selection using KBest with f_regreesion
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=selectkbest#sklearn.feature_selection.SelectKBest

In [ ]:
col_kbest = list(featureScores.nlargest(10,'Score')['Specs'])

### Time Series Regression
Four variables are selected and three are from stepwise regression. We added one variable: WDXRSA (S&P/Case-Shiller DC-Washington Home Price Index) based on business rationale  

In [ ]:
sel_x = train_x[['Days on Market_1diff', 'US_UR_1diff', 'New Listings MoM_1diff', 'WDXRSA_1diff']]
train_X = sm.add_constant(sel_x)
mod = sm.OLS(train_y, train_X, hasconst= True)
res = mod.fit()
print(res.summary())

#### Multicollinearity 

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(sel_x.values, i) for i in range(sel_x.shape[1])]
list(zip(['Days on Market_1diff', 'US_UR_1diff', 'New Listings MoM_1diff', 'WDXRSA_1diff'], vif))

#### Normality for the residual term: KS test and QQ plot

In [ ]:
from scipy.stats import kstest
from scipy.stats import stats
#residual = mlr.predict(X) - Y
residual = res.resid
kstest(residual, "norm")

In [ ]:
fig = sm.qqplot(residual, line = 'r')
plt.show()
fig.savefig("../data_visualization/Q-Q Plot for In-Sample Fit.png")

### Residual Analysis

In [ ]:
# %load ../python_folder/python_files/reverse_1st_diff

### In Time Sample

In [ ]:
pred_y = first_diff(data = res.predict(train_X), ini_value = df['Median Sale Price'][0])
error = df["Median Sale Price"][1:100] - pred_y[1:100]

#### Model Performance Metrics for In Time Sample
MAE: Around 10591.33 dollar impact 
    
MAPE: Around 4.02% price difference 

In [ ]:
#MAE
np.mean(abs(error))
#MAPE
np.mean(abs(error)/df["Median Sale Price"][1:100])

### OOT Sample 

In [ ]:
# %load ../python_folder/python_files/OOT_error

#### Model Performance Metrics for OOT Time Sample
MAE: Around 7882.99 dollar impact 
    
MAPE: Around 2.61% price difference 

In [ ]:
#MAE
np.mean((abs(error_oot)))
#MAPE
np.mean((abs(error_oot))/df["Median Sale Price"][100:117])

### Create a dataset for time series plot with Actual, In Time,and OOT samples

In [ ]:
# %load ../python_folder/python_files/data_predict

### Plot for Model Performance
We should the actual median house price in Blue, In Time Model Performance in Orange dash line, and OOT Model Performance in Green dot line

In [ ]:
list_data = [predict_m.loc[:, "Median Sale Price"], predict_m.loc[:"5 31 2018", "In Time"], predict_m.loc["6 1 2018":, "OOT Time"]]
ax = sns.lineplot(data=list_data, legend="full")
plt.title("Prediction House Price", fontsize = 40)
plt.xlabel("Month", fontsize = 30)
plt.ylabel("Condo Price", fontsize = 30)
plt.show()